### Elasticsearch Index Creation Example

In [15]:
from pprint import pprint
from elasticsearch import Elasticsearch

es=Elasticsearch("http://localhost:9200")
client_info = es.info()
print("Elasticsearch Client Information:")
pprint(client_info)


Elasticsearch Client Information:
ObjectApiResponse({'name': '82c3cca05ad1', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'PuXwAaOSSK-vVPeGsKp1QA', 'version': {'number': '8.15.0', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '1a77947f34deddb41af25e6f0ddb8e830159c179', 'build_date': '2024-08-05T10:05:34.233336849Z', 'build_snapshot': False, 'lucene_version': '9.11.1', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})


In [16]:
es.indices.create(index="my_index")

BadRequestError: BadRequestError(400, 'resource_already_exists_exception', 'index [my_index/AuQ3SaDiSvKrIy7K5T0khg] already exists')

### cerate an index with settings and mappings

# shard
#### shard is a basic unit of storage in Elasticsearch. It is a self-contained index that can be stored on any node in the cluster. Each shard is a Lucene index, and it can be thought of as a single index that can be searched independently.

# replica
#### A replica is a copy of a shard. It is used to provide redundancy and high availability
#### in case a node fails. Each shard can have multiple replicas, and each replica is stored
#### on a different node in the cluster. This allows Elasticsearch to continue serving requests even if a node fails.

In [18]:
es.indices.delete(index="my_index")
es.indices.create(index="my_index",
                  settings={
                      "number_of_shards": 3,
                      "number_of_replicas": 2,}
                  )


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'my_index'})

### create an index with mappings

In [19]:
document={
    "title": "Elasticsearch Basics",
    "content": "This is a basic introduction to Elasticsearch.",
    "author": "Abhishek Nangare",
    "published_date": "2023-10-01",
    "tags": ["elasticsearch", "search", "indexing"]

}

responce=es.index(index="my_index",body=document)
print("Document indexed successfully:")
pprint(responce)

Document indexed successfully:
ObjectApiResponse({'_index': 'my_index', '_id': 'ARWQbpgBHrgWYUmvirKj', '_version': 1, 'result': 'created', '_shards': {'total': 3, 'successful': 1, 'failed': 0}, '_seq_no': 0, '_primary_term': 1})


In [20]:
print(responce["result"])

created


In [23]:
print(responce["_shards"])

{'total': 3, 'successful': 1, 'failed': 0}


### insert multiple documents